# Fetch data about sensors and current polution level

In [1]:
import pandas as pd

In [2]:
import requests

headers = {
    'Accept': 'application/json',
    'apikey': '***',
}

#rybnik's geographical coordinates
params = (
    ('southwestLat', '50.033'),
    ('southwestLong', '18.428'),
    ('northeastLat', '50.180'),
    ('northeastLong', '18.662'),
)

response = requests.get('https://airapi.airly.eu/v1/sensors/current', headers=headers, params=params)

In [3]:
data_js = response.json()

## Convert json to dataframe

In [4]:
#only official rybnik's sensors
df_temp = pd.DataFrame.from_records(data_js).query("name == 'Rybnik' and vendor == 'Airly'")

#unfold address dict
df_temp = pd.concat([df_temp.drop(['address'], axis=1), df_temp['address'].apply(pd.Series)], axis=1)
#unfold location dict
sensors = pd.concat([df_temp.drop(['location'], axis=1), df_temp['location'].apply(pd.Series)], axis=1).reset_index(drop=True)

In [5]:
sensors

,id,name,pollutionLevel,vendor,country,locality,route,streetNumber,latitude,longitude
0,2083,Rybnik,3,Airly,Poland,"Rybnik, Chwałęcice",Gzelska 7,ZSP 15,50.133136,18.485427
1,2091,Rybnik,2,Airly,Poland,"Rybnik, Maroko-Nowiny",Orzepowicka 15a,ZS 3,50.100009,18.517341
2,2094,Rybnik,2,Airly,Poland,"Rybnik, Smolna",W. Reymonta 69,SP 34,50.090576,18.526136
3,2107,Rybnik,2,Airly,Poland,"Rybnik, Śródmieście",T. Kościuszki 5,ZST,50.096996,18.549377
4,2174,Rybnik,1,Airly,Poland,"Rybnik, Golejów",Komisji Edukacji Narodowej 29,ZSP 4,50.143863,18.540533
5,2198,Rybnik,2,Airly,Poland,"Rybnik, Ochojec",B. Kuglera 8a,ZSP 3,50.164684,18.546913
6,2202,Rybnik,2,Airly,Poland,"Rybnik, Paruszowiec-Piaski",Wolna 17,SP 3,50.096382,18.583908
7,2206,Rybnik,2,Airly,Poland,"Rybnik, Niewiadom",Sportowa 52,SP 23,50.074623,18.465013
8,2211,Rybnik,2,Airly,Poland,"Rybnik, Boguszowice Osiedle",J. Lompy 6,ZSP 8,50.048667,18.605903
9,2331,Rybnik,2,Airly,Poland,"Rybnik, Orzepowice",Borki 37D,ZSP 7,50.110969,18.516067


# List of sensors' ids

In [101]:
ids = list(sensors.id)

# Current - detaild data, 24 hours of history and 24 hours of forecast

In [110]:
import requests
from pandas.io.json import json_normalize

headers = {
    'Accept': 'application/json',
    'apikey': '7e6092f86c384ed7bf3bd561d13a931e',
}

allData = []
currentMeasure = []

for i, sensId in enumerate(ids):
    
    params = (
        ('sensorId', str(sensId)),
    )

    response = requests.get('https://airapi.airly.eu/v1/sensor/measurements', headers=headers, params=params)
    data_js = response.json()
    allData.append(pd.DataFrame.from_dict(json_normalize(data_js), orient='columns'))
    currentMeasure.append(allData[i].iloc[:,:-2])
    currentMeasure[i]['id'] = sensId

In [118]:
currentData = pd.concat(currentMeasure)

In [120]:
currentData = sensors.set_index('id').join(currentData.set_index('id'))

In [121]:
currentData

,name,pollutionLevel,vendor,country,locality,route,streetNumber,latitude,longitude,currentMeasurements.airQualityIndex,currentMeasurements.humidity,currentMeasurements.pm1,currentMeasurements.pm10,currentMeasurements.pm25,currentMeasurements.pollutionLevel,currentMeasurements.pressure,currentMeasurements.temperature
id,,,,,,,,,,,,,,,,,
2083,Rybnik,3,Airly,Poland,"Rybnik, Chwałęcice",Gzelska 7,ZSP 15,50.133136,18.485427,51.208800,74.950315,19.219200,56.245000,31.208800,3,100582.066699,0.905000
2091,Rybnik,2,Airly,Poland,"Rybnik, Maroko-Nowiny",Orzepowicka 15a,ZS 3,50.100009,18.517341,38.369333,85.000000,15.428000,34.818000,23.021600,2,100563.033276,0.900000
2094,Rybnik,2,Airly,Poland,"Rybnik, Smolna",W. Reymonta 69,SP 34,50.090576,18.526136,35.018667,72.127266,14.084000,31.311000,21.011200,2,100573.033386,1.947500
2107,Rybnik,2,Airly,Poland,"Rybnik, Śródmieście",T. Kościuszki 5,ZST,50.096996,18.549377,40.245333,72.290574,15.372000,38.094000,24.147200,2,100546.933435,0.845000
2174,Rybnik,1,Airly,Poland,"Rybnik, Golejów",Komisji Edukacji Narodowej 29,ZSP 4,50.143863,18.540533,29.162424,75.355732,12.269091,24.334545,17.497455,2,100581.276650,1.208977
2198,Rybnik,2,Airly,Poland,"Rybnik, Ochojec",B. Kuglera 8a,ZSP 3,50.164684,18.546913,29.052121,80.500000,12.426909,24.340909,17.431273,2,100556.140381,1.189318
2202,Rybnik,2,Airly,Poland,"Rybnik, Paruszowiec-Piaski",Wolna 17,SP 3,50.096382,18.583908,37.221333,73.443070,14.761600,35.917000,22.332800,2,100517.000012,1.325000
2206,Rybnik,2,Airly,Poland,"Rybnik, Niewiadom",Sportowa 52,SP 23,50.074623,18.465013,39.539394,77.871342,14.941818,37.914545,23.723636,2,100576.660500,1.457159
2211,Rybnik,2,Airly,Poland,"Rybnik, Boguszowice Osiedle",J. Lompy 6,ZSP 8,50.048667,18.605903,44.258667,85.000000,16.956800,43.344000,26.555200,2,100534.052466,1.320000


# 24 hours history

In [45]:
history = pd.pivot_table(pd.DataFrame.from_dict(df_temp.history[0][0]).reset_index(), index=['fromDateTime', 'tillDateTime'], columns = 'index', values='measurements')

for i in range(len(df_temp.history[0])-1):
    history = pd.concat([history, pd.pivot_table(pd.DataFrame.from_dict(df_temp.history[0][i+1]).reset_index(), index=['fromDateTime', 'tillDateTime'], columns = 'index', values='measurements')])
history

,index,airQualityIndex,humidity,pm1,pm10,pm25,pollutionLevel,pressure,temperature
fromDateTime,tillDateTime,,,,,,,,
2018-03-31T21:00:00Z,2018-03-31T21:59:59Z,48.049697,93.298174,17.680727,50.540000,28.829818,2.0,99600.775790,8.170000
2018-03-31T22:00:00Z,2018-03-31T22:59:59Z,38.733333,94.612163,15.260000,38.647000,23.240000,2.0,99503.266699,8.320000
2018-03-31T23:00:00Z,2018-03-31T23:59:59Z,56.887200,92.251103,21.946400,59.640000,36.887200,3.0,99551.966699,7.365000
2018-04-01T00:00:00Z,2018-04-01T00:59:59Z,57.783200,96.416171,22.904000,62.069000,37.783200,3.0,99571.766699,6.455000
2018-04-01T01:00:00Z,2018-04-01T01:59:59Z,46.302667,92.070175,17.668000,46.872000,27.781600,2.0,99543.966699,5.742500
2018-04-01T02:00:00Z,2018-04-01T02:59:59Z,47.489697,92.407878,18.164364,47.160909,28.493818,2.0,99567.866699,5.436364
2018-04-01T03:00:00Z,2018-04-01T03:59:59Z,51.787636,92.158451,19.783273,53.499091,31.787636,3.0,99629.684881,4.427273
2018-04-01T04:00:00Z,2018-04-01T04:59:59Z,48.782222,95.101923,18.523556,48.175556,29.269333,2.0,99679.644477,3.837778
2018-04-01T05:00:00Z,2018-04-01T05:59:59Z,51.561600,95.194305,19.728800,53.452000,31.561600,3.0,99753.966699,2.782500
